**Activation Maximization on C3D**

Let's use C3D model pretrained on Sports1M dataset

In [1]:
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Conv3D, MaxPool3D, ZeroPadding3D
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD

model = Sequential()

# 1st layer group
model.add(Conv3D(64, (3, 3, 3), activation='relu', padding='same', name='conv1', strides=(1, 1, 1), input_shape=(16, 112, 112, 3)))
model.add(MaxPool3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding='valid', name='pool1'))
# 2nd layer group
model.add(Conv3D(128, (3, 3, 3), activation='relu', padding='same', name='conv2', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool2'))
# 3rd layer group
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3a', strides=(1, 1, 1)))
model.add(Conv3D(256, (3, 3, 3), activation='relu', padding='same', name='conv3b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool3'))
# 4th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv4b', strides=(1, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool4'))
# 5th layer group
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5a', strides=(1, 1, 1)))
model.add(Conv3D(512, (3, 3, 3), activation='relu', padding='same', name='conv5b', strides=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='valid', name='pool5'))
model.add(Flatten())
# FC layers group
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(.5))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(.5))
model.add(Dense(487, activation='softmax', name='fc8'))
model.summary()

/home/olesia/test_env_3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 16, 112, 112, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 16, 56, 56, 64)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 16, 56, 56, 128)   221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 8, 28, 28, 128)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 8, 28, 28, 256)    884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 8, 28, 28, 256)    1769728   
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 4, 14, 14, 256)    0         
__________

Lets load wights and compile the model

In [ ]:
model.load_weights('sports1M_weights_tf.h5')
model.compile(loss='mean_squared_error', optimizer='sgd')

Getting indices of convolutional layers

In [2]:
layer_names = ['conv1', 'conv2', 'conv3a', 'conv3b', 'conv4a', 'conv4b', 'conv5a', 'conv5b']
layer_indices = []
for name in layer_names:
  layer_indices.append([idx for idx, layer in enumerate(model.layers) if layer.name == name][0])
layer_indices

[0, 2, 4, 5, 7, 8, 10, 11]

Getting number of filters for each convolution layer

In [4]:
from vis.utils import utils
import numpy as np
from vis.visualization import visualize_activation, get_num_filters

number_of_filters = []
for idx in layer_indices:
  number_of_filters.append(get_num_filters(model.layers[idx]))
number_of_filters

SyntaxError: invalid syntax (<ipython-input-4-347d53faaf3d>, line 1)